# Anthropic Websearch Tool

A simple examlpe of using Anthropic's [Websearch tool](https://docs.anthropic.com/en/docs/build-with-claude/tool-use/web-search-tool)

## Usage
Add your anthropic API key to `.env`: `ANTHROPIC_API_KEY=<your api key>`

In this example, we ask Claude about the Pope - one of the most substantial current events. 

Since Claude's training cutoff is long before Pope Francis Died and Leo was elected, so it is unaware of the recent events.

In essence, this is an example RAG pipeline using web search instead of using vectors or graphs.

In [1]:
import anthropic
import json

from dotenv import load_dotenv

load_dotenv()

client = anthropic.Anthropic()

messages = [
    { "role": "user",
      "content": "What can you tell me about the Pope?"
    }
]

## Make request to Claude 3.7 Sonnet as a benchmark.

In [2]:
non_search_response = client.messages.create(
    model="claude-3-7-sonnet-latest",
    max_tokens=100,
    messages=messages,
)

print(json.dumps(non_search_response.model_dump(), indent=4))


{
    "id": "msg_014MdPgqgfeooFqgUsJDqGnv",
    "content": [
        {
            "citations": null,
            "text": "The Pope is the Bishop of Rome and the head of the Catholic Church worldwide. The current Pope is Francis (born Jorge Mario Bergoglio), who became the 266th Pope in 2013 after Pope Benedict XVI's resignation. He's the first Jesuit Pope and the first from the Americas.\n\nThe Pope lives in Vatican City, a sovereign city-state within Rome. The position carries significant religious authority for Catholics, who believe the Pope is the successor to Saint Peter an",
            "type": "text"
        }
    ],
    "model": "claude-3-7-sonnet-20250219",
    "role": "assistant",
    "stop_reason": "max_tokens",
    "stop_sequence": null,
    "type": "message",
    "usage": {
        "cache_creation_input_tokens": 0,
        "cache_read_input_tokens": 0,
        "input_tokens": 16,
        "output_tokens": 100,
        "server_tool_use": null
    }
}


As suspected, Claude is unaware of the current events.

## Make the same call using the `web_search` tool

In [3]:
web_search_response = client.messages.create(
    model="claude-3-7-sonnet-latest",
    max_tokens=100,
    messages=messages,
    tools=[{
        "type": "web_search_20250305",
        "name": "web_search",
        "max_uses": 5
    }]
)


print(json.dumps(web_search_response.model_dump(), indent=4))

{
    "id": "msg_01SEW84RgQrCUiTMMX9LiQgw",
    "content": [
        {
            "citations": null,
            "text": "I'd be happy to provide information about the Pope. Let me search for the most current information for you.",
            "type": "text"
        },
        {
            "id": "srvtoolu_01WE2aDaMMEVovwMAsGg67dn",
            "input": {
                "query": "Pope Francis current status information Catholic leader May 2025"
            },
            "name": "web_search",
            "type": "server_tool_use"
        },
        {
            "content": [
                {
                    "encrypted_content": "EuwfCioIAxgCIiQwZDFjN2Y4Mi04NDI5LTRiYWYtYjYwYi1iYmNmMjRmNzM1YTUSDFXkKDRceWIQnK6yLhoMJ31WY1TyHV5bW/SRIjDnxmeDpmN+KL/ueeykV15rDPpJlME71ScATjLp4RMLa3bJ6VG4fYeQOoknJxSU7twq7x4Um/dnL6/e93XinMlzR01Q1ZbaWUPDZb7Va2njdcNiwLDLP26CBhQImIZwYGBCPF0dsLZkbXwqsOPrFs96lo27IAa2FmBIXI9oiCaq0TvWd610e4UHHkt1xEmd0tODIKytSqp2hSS6KhUe+i7s9ezxWTyKNbmLzVa8YagrSCxJujfgNHZszb4p68aR

We can see that the `web_search` tool returned 10 resutls containing the the `encrypted_content`, `page_age`, `title`, `type` and `url` from the performed search as an array in the content property of the response. We pass this array as the `assistant` message back to Claude to generate a summary of the content.

In [4]:
messages.append({'role': 'assistant', 'content': web_search_response.content})
messages.append({
    "role": "user",
    "content": "summarize the result"
})

final_response = client.messages.create(
    model="claude-3-7-sonnet-latest",
    max_tokens=1024,
    messages=messages,
)


print(json.dumps(final_response.model_dump(), indent=4))

{
    "id": "msg_01HJHNELrB56JK8anfqXYiNu",
    "content": [
        {
            "citations": null,
            "text": "Here's a summary of the current Pope situation:\n\nPope Francis passed away on April 21, 2025, at the age of 88. Following his death, a conclave of cardinals gathered at the Vatican to elect his successor.\n\nOn May 8, 2025, Cardinal Robert Francis Prevost was elected as the 267th pope of the Catholic Church. He has taken the name Pope Leo XIV and is the first American-born pope in history. Cardinal Prevost, who is 69 years old, was born in Chicago but spent much of his career as a missionary in Peru, where he also holds citizenship.\n\nPrior to becoming pope, Prevost was an Augustinian friar who most recently served as the prefect of the Dicastery for Bishops in the Vatican. He was elevated to cardinal by Pope Francis in 2023.\n\nThis historic election represents a significant moment for the Catholic Church, as Leo XIV is expected to continue many of Francis's ref

## Print the final results

In [5]:
for content in final_response.content:
    print(content.text)

Here's a summary of the current Pope situation:

Pope Francis passed away on April 21, 2025, at the age of 88. Following his death, a conclave of cardinals gathered at the Vatican to elect his successor.

On May 8, 2025, Cardinal Robert Francis Prevost was elected as the 267th pope of the Catholic Church. He has taken the name Pope Leo XIV and is the first American-born pope in history. Cardinal Prevost, who is 69 years old, was born in Chicago but spent much of his career as a missionary in Peru, where he also holds citizenship.

Prior to becoming pope, Prevost was an Augustinian friar who most recently served as the prefect of the Dicastery for Bishops in the Vatican. He was elevated to cardinal by Pope Francis in 2023.

This historic election represents a significant moment for the Catholic Church, as Leo XIV is expected to continue many of Francis's reforms while bringing his own perspective as the first North American pontiff.
